In [1]:
from numpy import dtype
import random
import operator
import csv
import itertools
import numpy
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from Source.code import *
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
df = le_arq("./Data/telecom_users.csv")
le = preprocessing.LabelEncoder()
for i in df:
    if(dtype(df[i]) == 'object'):
        df[i] = le.fit_transform(df[i])

In [3]:
spam=df.to_numpy().tolist()
print(df.info())
train, test = train_test_split(spam, random_state=42)
# defined a new primitive set for strongly typed GP
pset = gp.PrimitiveSetTyped("MAIN", list(itertools.repeat(int, 17)) + list(itertools.repeat(float, 2)), bool, "IN")
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5974 entries, 0 to 5985
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Genero                  5974 non-null   int64  
 1   Aposentado              5974 non-null   int64  
 2   Casado                  5974 non-null   int64  
 3   Dependentes             5974 non-null   int64  
 4   MesesComoCliente        5974 non-null   int64  
 5   ServicoTelefone         5974 non-null   int64  
 6   MultiplasLinhas         5974 non-null   int64  
 7   ServicoInternet         5974 non-null   int64  
 8   ServicoSegurancaOnline  5974 non-null   int64  
 9   ServicoBackupOnline     5974 non-null   int64  
 10  ProtecaoEquipamento     5974 non-null   int64  
 11  ServicoSuporteTecnico   5974 non-null   int64  
 12  ServicoStreamingTV      5974 non-null   int64  
 13  ServicoFilmes           5974 non-null   int64  
 14  TipoContrato            5974 non-null   

,Genero,Aposentado,Casado,Dependentes,MesesComoCliente,ServicoTelefone,MultiplasLinhas,ServicoInternet,ServicoSegurancaOnline,ServicoBackupOnline,ProtecaoEquipamento,ServicoSuporteTecnico,ServicoStreamingTV,ServicoFilmes,TipoContrato,FaturaDigital,FormaPagamento,ValorMensal,TotalGasto,Churn
0,1,0,1,1,72,1,2,2,1,1,1,1,1,1,0,0,2,24.10,1734.65,0
1,0,0,0,0,44,1,0,1,0,2,2,0,2,0,2,1,2,88.15,3973.20,0
2,0,1,1,0,38,1,2,1,0,0,0,0,0,0,2,1,3,74.95,2869.85,1
3,1,0,0,0,4,1,0,0,0,0,0,0,0,2,2,1,0,55.90,238.50,0
4,1,0,0,0,2,1,0,0,2,0,2,0,0,0,2,0,0,53.45,119.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5981,1,0,1,0,1,1,0,1,2,0,0,0,2,2,2,1,0,95.00,95.00,1
5982,0,0,1,1,23,1,2,0,2,2,2,2,2,2,0,1,2,91.10,2198.30,0
5983,1,0,1,1,12,1,0,2,1,1,1,1,1,1,2,1,0,21.15,306.05,0
5984,1,1,0,0,12,1,2,1,0,0,2,0,2,2,2,1,0,99.45,1200.15,1


In [4]:
def protectedDiv(left, right):
    try: return left / right
    except ZeroDivisionError: return 1

def if_then_else(input, output1, output2):
    if input: return output1
    else: return output2

def evalSpambaseTrain(individual):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Randomly sample 400 mails in the spam database
    train_samp = random.sample(train, 400)
    # Evaluate the sum of correctly identified mail as spam
    result = sum(bool(func(*mail[:19])) is bool(mail[19]) for mail in train_samp)
    return result, 

In [5]:
# boolean operators
pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.not_, [bool], bool)
pset.addPrimitive(operator.add, [float, float], float)
pset.addPrimitive(operator.sub, [float, float], float)
pset.addPrimitive(operator.mul, [float, float], float)
pset.addPrimitive(protectedDiv, [float, float], float)
pset.addPrimitive(operator.lt, [float, float], bool)
pset.addPrimitive(operator.eq, [float, float], bool)
pset.addPrimitive(if_then_else, [bool, float, float], float)
# terminals
pset.addEphemeralConstant("rand100", lambda: random.random() * 100, float)
pset.addTerminal(False, bool)

Exception: Ephemerals with different functions should be named differently, even between psets.

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=5, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSpambaseTrain)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [ ]:
def main():
    random.seed(10)
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)

    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats,
                        halloffame=hof, verbose=None)

    return hof


In [ ]:
for i in range(30):    
        tree = main()[0]
        #tree = tree[0]
        func = toolbox.compile(expr=tree)
        result_test = sum(bool(func(*customer[:19])) is bool(customer[19]) for customer in test) / len(test)
        result_train = sum(bool(func(*customer[:19])) is bool(customer[19]) for customer in train) / len(train)
        print(i,"-> Result Test: ", result_test)
        print(i,"-> Result Train: ",result_train)
        print(i,"-> Hof: ",tree)